In [ ]:
import numpy as np
from qat.lang import qrout, H, CNOT, X
from qat.lang.AQASM import Program
from qat.lang.AQASM.qftarith import QFT
from qat.qpus import get_default_qpu

# Criptoanálise contra Criptografia Assimétrica

Criptografia Assimétrica é o nome dado ao esquema criptográfico que envolve duas
chaves, comumente chamadas de chave pública e chave privada.  Nesse esquema
criptográfico, todo dado encriptado por uma das chaves só pode ser decifrado com
o uso da chave complementar.

Esse tipo de criptografia é amplamente usado em comunicações pela internet.
Nesse caso, o usuário irá gerar um par de chaves, das quais uma ele irá publicar
(broadcast) e a outra ele irá manter em sigilo, daí os nomes chave pública e
chave privada. Qualquer pessoa que deseja se comunicar com esse usuário poderá
encriptar sua mensagem usando a chave pública disponível e terá a garantia que
apenas o usuário poderá ler a mensagem. 

Um criptossistema de chave pública é definido por uma quíntupla ($\mathcal{P}, \mathcal{C}, \mathcal{K}, \mathcal{E}, \mathcal{D}$), onde as sete condições a seguir são satisfeitas:

 1. $\mathcal{P}$ é um conjunto finito de *textos simples* possíveis.
 2. $\mathcal{C}$ é um conjunto finito de *textos cifrados* possíveis.
 3. $\mathcal{K}$ é um conjunto finito de chaves possíveis. $\mathcal{K}$ é chamado de *espaço de chave* (*keyspace*).
 4. Para cada chave no keyspace $K \in \mathcal{K}$, existe uma regra de criptografia $enc_K \in \mathcal{E}$ e uma regra de descriptografia correspondente $dec_K \in \mathcal{D}$. Cada $enc_K : \mathcal{P} \rightarrow \mathcal{C}$ e $dec_K : \mathcal{C} \rightarrow \mathcal{P}$ são funções tais que $dec_K (enc_K (m)) = m$ para todo texto simples $m \in \mathcal{P}$.
 5. Para cada chave $K \in \mathcal{K}$ e cada texto simples $m \in \mathcal{P}$, tanto $enc_K (m)$ quanto $dec_K (enc_K (m))$ são fáceis de calcular.
 6. Para quase todas as chaves $K \in \mathcal{K}$, cada algoritmo facilmente computável equivalente a $dec_K$ é computacionalmente inviável para derivar de $enc_K$. Ou seja, é difícil descriptografar sem $dec_K$.
 7. A regra de criptografia $enc_K$ é tornada pública e a regra de descriptografia $dec_K$
 é mantido em sigilo.


## Algoritmo de Shor vs RSA

O criptossistema RSA, em homenagem aos seus inventores Ron Rivest, Adi Shamir e
Leonard Adleman, é o criptossistema de chave pública mais conhecido e usado no
mundo atualmente.

Por exemplo, como um dos sistemas criptográficos de chave pública usados no
protocolo Transport Layer Security (TLS) e seu antecessor, o protocolo Secure
Sockets Layer (SSL), o sistema criptográfico RSA é usado milhões de vezes por
dia na Internet. Essencialmente, o RSA é usado para transmitir uma chave de
sessão para um sistema criptográfico simétrico que é então usado para garantir a
segurança de uma comunicação.


### Algoritmo RSA

Seja $N = pq$ o produto de dois números primos $p$ e $q$, e definindo
$\mathcal{P} = \mathcal{C} = \mathbb{Z}_N$ (conjunto dos inteiros módulo $N$) e o espaço de chaves como

$$\mathcal{K} = \{ (N,p,q,e,d) \>:\> ed \equiv 1 (\mod{\phi(N)}) \},$$

onde $\phi(N) = (p-1)(q-1)$ é a função Totiente de Euler. Para cada chave $K \in \mathcal{K}$, dada por $K = (N,p,q,e,d)$, a regra de criptografia $enc_K\>:\>\mathbb{Z}_N \rightarrow \mathbb{Z}_N$ é definida por

$$enc_K(x) = x^e \mod{N},$$ 

e a regra de descriptografia $dec_K\>:\>\mathbb{Z}_N \rightarrow \mathbb{Z}_N$ é definida por

$$dec_K(y) = y^d \mod{N}$$

para todo $x,y \in \mathbb{Z}$. o par $(e,N)$ é a chave pública do RSA e a tripla $(d,p,q)$ é a chave privada do RSA.

Já que os expoentes público e privado $e,d$ devem satisfazer a relação

$$ed \equiv 1 (\mod(\phi(N))),$$

por consequência

$$ed = 1 + k\phi(N)$$

Tendo acesso à chave pública $(e, N)$ e fatorando $N$ em seus componentes $p$ e $q$ é possível recalcular o expoente privado $d$ e obter a chave privada $(d,p,q)$ a partir dessa relação.

In [ ]:
# Implementação em Python do algoritmo RSA. Por demonstração, os valores usados
# abaixo estão muito abaixo dos usados em prática

# Escolha do Par de Números Primos 
p = 3
q = 5
N = p*q

### Geração da Chave Pública
e = 2
phi = (p-1)*(q-1)
while (e < phi):
    # e precisa ser co-primo e inferior a phi
    if(np.gcd(e, phi) == 1):
        break
    else:
        e = e+1
 
### Geração da Chave Privada
# Escolha de um valor que satisfaça d*e = 1 + k * phi
k = 2
d = (1 + (k*phi))/e

public_key = (e, N)
private_key = (p,q,k)

print("A chave pública é:", public_key)
print("A chave privada é:", private_key)

# Mensagem a ser encriptada
plaintext = 2.0
print("Message data = ", plaintext)
 
# Encryption c = (msg ^ e) % n
ciphertext = np.power(plaintext, e)
ciphertext = np.fmod(ciphertext, N)
print("Encrypted data = ", ciphertext)
 
# Decryption m = (c ^ d) % n
message = np.power(ciphertext, d)
message = np.fmod(message, N)
print("Original Message Sent = ", message)

### Algoritmo de Shor parte 1: Circuito Quântico

O Algoritmo de Shor pode ser utilizado para encontrar fatores de um número
inteiro, $N$, sendo $N = pq$ onde $p$ e $q$ são números primos. 

O algoritmo usa dois registradores, o primeiro registrador tem $n = \lceil
\log_2 N^2 \rceil = \lceil 2 \log_2 N \rceil$ qubits e possui os qubits de
medição. O segundo registrador tem $m = \lceil \log_2 N \rceil$ qubits, e é o
autoestado para a estimativa da fase quântica. O processo ocorre em quatro
etapas principais, semelhantes à implementação da estimativa de fase quântica. 

1. Inicialização dos qubits. Cria-se uma superposição de todos os $2^n$ estados
da base computacional nos $n$ qubits do primeiro registrador, aplicando uma
porta Hadamard ($H$) em cada qubit. Também inicializamos os  $m$ qubits do
segundo registrador  no estado $\vert1\rangle=\ket{00\ldots1}$. 

2. Aplicação do operador unitário $U_a|x\rangle = |ax \mod N\rangle $, que
realiza a exponenciação modular no segundo registrador. Esse operador é
controlado por cada um dos qubits do primeiro registrador. O esquema abaixo
mostra a ordenação e as respectivas potências.

<p align="center">
<img src="imagens/shorimage.png" width=500 align=center/>
</p>

3. Aplicação da transformada de Fourier quântica inversa nos qubits do primeiro
registrador.

4. Medida dos qubits do primeiro registrador, colapsando para o estado
$|l\rangle$ .


## Exercício hands-on

Substitua os `FIXME`s pelas respostas corretas.

### Defina o número de qubits em cada registrador

Leia a descrição do Algoritmo de Shor para obter a resposta para o valor de cada
registrador, sendo $n$ o número de qubits no primeiro registrador e $m$ o do
segundo.

Para o número $15$, podemos usar $n=m=\lceil \log_2 N \rceil$, visto que a ordem é sempre uma potência
de $2$.

In [ ]:
print(f"Número a ser fatorado {N}")

# numero de qubits no primeiro registrador.
n = FIXME

# numero de qubits no segundo registrador.
m = FIXME

# numero total de qubits
num_qubits = n+m

print(f"{n} qubits no registrador de medida")
print(f"{m} qubits no segundo registrador")


#### Encontre possíveis valores de $a$

Esse valor será usado para calcular a exponenciação modular $a^r\mod15$. Note que ele deve ser coprimo com $N$, ou seja, $MDC(a,N)=1$, e deve ser menor que $N$.

Dica: Use np.gcd() para calcular o MDC de $a$ e $n$

Dica²: Você pode usar um laço de repetição para gerar diferentes valores possíveis

In [ ]:
# Escreva seu código abaixo

a = FIXME

### Inicializando os qubits

Inicializamos o registrador de medida como uma superposição uniforme através da operação $H^{\otimes n}|0\rangle^{\otimes n}$. O segundo registrador, onde será aplicada a exponenciação modular, é inicializado no estado $|1\rangle \equiv |00...001\rangle$.

Logo, o estado inicializado, $|\psi_0\rangle$, é dado por

$$|\psi_0\rangle = \frac{1}{\sqrt{2^n}}\sum_{i=0}^{2^n-1}|i\rangle\otimes |1\rangle$$

DICA: Escreva a inicialização de qubits usando uma QRoutine do QLM. Uma rotina pode ser criada de duas formas:

    - Usando a *label* @qrout no início de uma função
    - Declarando o objeto QRoutine()

DICA²: Não esqueça de visualizar o seu circuito

DICA³: Modifique a assinatura da função para que ela receba qualquer parâmetro que julgue necessário

In [ ]:
@qrout
def initialize_qubits():
    pass

# visualizando o circuito de inicialização
initialize_qubits().display()

### Exponenciação Modular Quântica

Nessa etapa, precisamos gerar os valores da exponenciação modular $a^{2^i} \mod 15$ no segundo registrador, e para cada inteiro $i$ será associado um qubit no primeiro registrador que irá controlar esta porta.

Para tal, precisamos implementar várias vezes o operador: $$U_a|y\rangle = |a y\mod 15\rangle$$

Usaremos como exemplo, a implementação usada em https://arxiv.org/pdf/1202.6614.pdf para $a=7$, dado pelo circuito:

<p align="center">
<img src="imagens/M7.jpg" width=200 />
</p>

Para cada $i$, basta aplicarmos este circuito $2^i$ vezes de forma controlada a partir do valor inicial 1.

O circuito acima pode ser controlado extendendo as portas $X$ para CNOTS com um qubit de controle e abrindo a porta SWAP em sua decomposição por CNOTs. Assim, cada CNOT pode ser extendida para uma porta Toffoli (CCNOT) com o qubit de controle. Vide imagens abaixo:

<p align="center">
<img src="imagens/M7Aberto.jpg" width=400 />
</p>

<p align="center">
<img src="imagens/CM7.jpg" width=400 />
</p>

Para generalizar esta ideia com um $a$ diferente de 7, basta obter a representação do $U_a$ em CNOTS e SWAPS e convertê-lo para uma versão controlada. O material de referência mostra circuitos para outros valores de $a$.

OBS: Faça o circuito para $a=8$ e, caso sobre tempo, para $a=14$. 

OBS²: Acompanhe o que foi feito para $a=7$ e, baseado na referência dada, reproduza de forma análoga para os outros valores de $a$.


In [ ]:
@qrout
def controlled_modular_exponentiation(i):
    pass

# visualizando a exponenciação modular
controlled_modular_exponentiation(1).display()

### Criando o programa

Neste exercício, você terá que aplicar todos os blocos de operações do Algoritmo de Shor no circuito quântico.
O programa consiste na aplicação das três principais operações descritas acima:

1. Inicialização do circuito
2. Exponenciação modular
3. Transformada Quântica de Fourier (QFT) no primeiro registrador

As medidas serão feitas e usadas no pós-processamento a fim de se encontrar o período $r$.

DICA: Use a função de QFT do próprio QLM.

In [ ]:
# criando o objeto shor com a classe Program
shor = Program()

### Escreva seu código abaixo
# alocando qubits no programa shor

# Aplique a rotina de inicialização

# Aplique a Exponenciação Modular

# Aplique a QFT no primeiro registrador


# transformando de programa para circuito 
circuit = shor.to_circ()

# visualizando o circuito
# Obs: na interface web do jupyter é possível dar zoom na imagem e visualizar melhor o circuito
%qatdisplay circuit --svg

### Realizando uma única medida no primeiro registrador

In [ ]:
# submeta o circuito para job
# Repare que será realizada apenas uma medida no circuito 
job = circuit.to_job(nbshots=1, qubits=FIXME)
result = get_default_qpu().submit(job)

# Aqui imprimimos os estados medidos e suas respectivas probabilidades
# A lista states armazenará todos os estados medidos (ou apenas um unico estado se nbshots=1) para pos processamento
states = []
for sample in result:
     print("State %s: probability %s " % (sample.state, sample.probability))
     states.append(sample.state)

### Transformando o valor medido para decimal

Nesta etapa, vamos transformar o valor medido para decimal. A lista *states* criada armazena todas as amostras obtidas, para acessar o valor de uma amostra pode se acessar o atributo sample.value. O QLM armazena os valores dos estados dos qubits em notação *little-endian*, portanto para gerar o valor decimal final é preciso reverter a string que representa o estado.

In [ ]:
states[0].value

In [ ]:
# O codigo abaixo converte os estados medidos no primeiro registrador em decimais
# Criamos uma lista string onde cada elemento corresponde a '0's ou '1's medidos no primeiro registrador 
string_reg1 = states[0].value[0]
print("Um valor binário no 1o registrador: ", string_reg1)


l = int(''.join(string_reg1[::-1]),2)
print("Valor decimal medido, l =  ", l)

### Algoritmo de Shor parte 2: Pós-processamento Clássico

Depois que os resultados da medição forem determinados, é necessário fazer um
pós-processamento clássico adicional para determinar os fatores ou decidir
executar o programa novamente.

1. Calcula-se $\frac{l}{2^n}$ e toma-se o denominador como candidato ao período $r$. 

2. Testar se $r$ é período. Se $ a^r = 1 \mod N$, então $r$ é o período e é garantido que os fatores primos de $N$ serão obtidos através da relação

$$p,q = \text{mdc}(a^{r/2} \pm 1, N)$$

### Algoritmo de frações continuadas

A expansão em uma fração continuada de $\frac{\ell}{2^n}$, onde $\ell$ é a saída
do algoritmo, é dada por $$\frac{\ell}{2^n}=\cfrac{1}{a_1 + \cfrac{1}{a_2 +
\cfrac{1}{ \ddots + \cfrac{1}{a_k} }}}$$ Os sucessivos convergentes da fração
continuada são $[a_1]$, $[a_1,a_2]$, $[a_1,a_2,a_3]$ e assim por diante onde

$$[a_1]=\cfrac{1}{a_1},\;\;[a_1,a_2]=\cfrac{1}{a_1 + \cfrac{1}{a_2
}},\;\;[a_1,a_2,a_3]=\cfrac{1}{a_1 + \cfrac{1}{a_2 + \cfrac{1}{ a_3}}},
\;\;\cdots$$ 

O último convergente é $[a_1,a_2,...,a_k]$, que é igual a $\frac{\ell}{2^m}$. No
algoritmo de Shor, temos que escolher o convergente mais próximo de
$\frac{\ell}{2^m}$ tal que o denominador seja menor do que $N$.  Por exemplo, os
convergentes da expansão em fração continuada de $\ell/2^m$ para $\ell=85$ e
$m=9$ são $[6]=\frac{1}{6}$, $[6,42]=\frac{42}{253}$, e por último
$[6,42,2]=\frac{85}{512}=\ell/2^m$. 

[//]: # (Quando $N=21$, temos que selecionar o convergente $[6]=\frac{1}{6}$.)

O valor decimal medido é dado por $l$. Implemente abaixo o algoritmo de frações continuadas em $l/2^n$ para encontrar o candidato a $r$. Verifique se esse $r$ satisfaz $a^r \equiv 1\mod15$. Caso contrário, você terá que rodar o algoritmo novamente, realizando uma nova medida.

DICA: utilize o pacote do PyPi (https://pypi.org/project/ContFrac/) para implementar o algoritmo de frações continuadas, encontrar os convergentes e o valor candidado a ordem $r$. Você também pode implementar seu próprio código.

In [ ]:
r = 0
### Implemente seu código abaixo 


print("Candidato a r: ", r)
# NOTA: se voce encontrar r ímpar, terá de rodar o algoritmo novamente

### Determinando os fatores primos de $N$

Use $r$ para encontrar os fatores primos de $N$.

Dica: voce pode usar a função no numpy, np.gcd() para facilitar o cálculo do MDC.

In [ ]:
p = 0
q = 0

### Escreva seu codigo abaixo 


print(f"Os fatores primos encontrados são {p} e {q}.")

Se você não encontrou os fatores primos corretos, significa  que a medida que voce realizou não te forneceu a ordem $r$ correta. Você terá que rodar o algoritmo uma quantidade suficiente de vezes para encontrar o $r$ adequado. Você também pode aumentar o número de medidas `nbshots` e criar uma rotina que testa todos os valores medidos. Caso prefira, apenas rode o notebook mais algumas vezes.

### Recuperando a chave privada

In [ ]:
phi = (p-1)*(q-1)

# Escolha de um valor que satisfaça d*e = 1 + k * phi
k = 2
d = (1 + (k*phi))/e

print(f"A chave privada é ({p}, {q}, {d})")